This tutorial will guide you through the process of finding the <i><u>equilibrium</u></i> state of a mixture in <strong>PyChemkin</strong>. The <em>Mixture</em> <code>equilibrium</code> method with the <u>constant pressure and enthalpy</u> option is employed to estimate the adiabatic flame temperature of a methane-oxygen mixture. The influence of equivalence ratio on the predicted adiabatic flame temperature will be explored.

To start a <strong>PyChemkin</strong> project, you always have to import these Python packages: <code>os</code>, <code>chemkin</code>, and <code>numppy</code>. The <code>matplotlib</code> package is also imported for plotting purposes.

In [ ]:
import os
import chemkin as ck                      # Chemkin
import numpy as np                        # number crunching
%matplotlib inline
import matplotlib.pyplot as plt           # plotting

# check working directory
current_dir = os.getcwd()
print("current working directory: " + current_dir)
# set verbose mode
ck.setverbose(True)

<h5>Set up the <em>Chemistry Set</em></h5>Create and preprocess the <em>Chemistry</em> set object for the current PyChemkin project. Here the GRI 3.0 mechanism for methane combustion is used to create the <code>MyGasMech</code> <em>Chemistry</em> object. You do not need to specify the transport data file since the equlibrium calculation do not involve transport properties. 

In [ ]:
# set mechanism directory (the default chemkin mechanism data directory)
data_dir = os.path.join(ck.ansys_dir, "reaction", "data")
mechanism_dir = data_dir
# create a chemistry set based on the diesel 14 components mechanism
MyGasMech = ck.Chemistry(label="GRI 3.0")
# set mechanism input files
# inclusion of the full file path is recommended
MyGasMech.chemfile = os.path.join(mechanism_dir, "grimech30_chem.inp")
MyGasMech.thermfile = os.path.join(mechanism_dir, "grimech30_thermo.dat")

iError = MyGasMech.preprocess()

<h5>Set up the CH4-O2 mixture</h5>Use the <i><u>equivalence ratio</u></i> method to set up the combustible mixture so that you can easily change the mixture composition by assigning a different equivalence ratio value.

Set up the O2 "mixture".

In [ ]:
oxid = ck.Mixture(MyGasMech)
# set O2 mole fraction in the oxid "mixture"
oxid.X = [("O2", 1.0)]
oxid.temperature = 295.15
oxid.pressure = ck.Patm                           # 1 atm

Set up the CH4 "mixture".

In [ ]:
fuel = ck.Mixture(MyGasMech)
# set CH4 mole fraction in the fuel "mixture"
fuel.X = [("CH4", 1.0)]
fuel.temperature = oxid.temperature
fuel.pressure = oxid.pressure

Specify the complete combustion products from the CH4-O2 mixture. Use a all-zero array <code>add_frac</code> for the additive composition.

In [ ]:
# declare the complete combustion products
products = ["CO2", "H2O"]
# set up a all-zero array to indicate no additive
add_frac = np.zeros(MyGasMech.KK, dtype=np.double)

Create the CH4-O2 mixture <code>FAmixture</code> and assign its initial pressure and temperature. Note that the initial composition of <code>FAmixture</code> has not been defined yet. 

In [ ]:
# create the mixture 
FAmixture = ck.Mixture(MyGasMech)
# specify the mixture pressure and temperature (the same as the oxid mixture)
FAmixture.pressure = oxid.pressure
FAmixture.temperature = oxid.temperature

<h5>Calculate the adiabatic flame temperature</h5>Use the <code>equilibrium</code> method to compute the adiabatic flame temperature of the CH4-O2 mixture of given equivalence ratio.  

Set up the variable arrays to be plotted later.  

In [ ]:
# create the variable arrays for plotting
points = 12
# flame temperature
T = np.zeros(points, dtype=np.double)
# equivalence ratio
equiv = np.zeros_like(T, dtype=np.double)

The initial composition of the <code>FAmixture</code> is assigned by using the <em>Mixture</em> method <code>XbyEquivalenceRatio</code> with the desired equivalence ratio value. The the mixture <u>equivalence ratio</u> from 0.5 to 1.5. Set the option value of the <code>equilibrium</code> method to 5, i.e.,<code>opt=5</code>, to specify the <i><u>constant pressure and enthalpy</u></i> constraints for the equilibrium calculation. The <code>equilibrium</code> method returns a mixture object <code>EQState</code> representing the mixture state at equilibrium. You can get the equilibrium temperature using <code>EQState.temperature</code> method.<hr>You can find out all the options of the <i><u>equilibrium</u></i> method by typing <code>ck.help(topic='equilibrium')</code> at command line under the Pythone environment. 

In [ ]:
ck.help(topic="equilibrium")

In [ ]:
# initial mixture equivalence ratio value
equiv_ini = 0.5
# increment 
deq = 0.1
# set error flag
iError = 0
# increase the mixture equivalence ratio from 0.5 to 1.5 
for i in range(points):
    equiv_current = equiv_ini
    # define the composition of the initial state 
    iError = FAmixture.XbyEquivalenceRatio(MyGasMech, fuel.X, oxid.X, add_frac, products, equivalenceratio=equiv_current)
    if iError > 0:
        raise RuntimeError
    # the equilibrium method return a mixture object representing the equilibrium state
    EQState = ck.equilibrium(FAmixture, opt=5)
    # get the mixture temperature of the equilibrium state
    T[i] = EQState.temperature
    equiv[i] = equiv_current
    equiv_ini += deq

<h5>Display the results</h5>Plot the predicted adiabatic flame temperature against the equivalence ratio of the CH4-O2 mixture.

In [ ]:
# plot results
plt.plot(equiv,T,"bs--")
plt.xlabel("Equivalence ratio")
plt.ylabel("Temperature [K]")
plt.show()